In this Excercise we will work with Ray Tune and explore different options to optimize network hyperparametrs.

Ray Tune:
===========

Tune is a framework/library for distributed hyper-parameter search. Its purpose is to link an optimization algorithm 
and a trial scheduler together to run asynchronous trials.


<b>Basic Concepts:- </b>

1) <b>Search space:- </b>
define hyperparameters to tune in <br>
2) <b>Trainable:- </b>
Objective function to tune. Search space is passed to this objective function <br>
3) <b>Search Algorithm:- </b>
Algorithm which effectively optimize the parameters <br>
4) <b>Scheduler:- </b>
It is optional. It is used for early stopping and speed up your experiment <br>
5) <b>Trials:- </b>
Trainable, Search Algorithm and Scheduler are passed to Tuner which runs the experiment and creats trails. <br>
6) <b>Analyses:- </b>
Trails are passed to analyses to inspect the experiment results. <br>


Search space \ 
 >             \--> Trainable \ 
 >                             \ 
 >           Search Algorithm    -- > Trails --> Analyses 
 >                             / 
 >                  Scheduler / 


### Search space

config = { <br>
    "uniform": tune.uniform(-5, -1),  # Uniform float between -5 and -1 <br>
    "quniform": tune.quniform(3.2, 5.4, 0.2),  # Round to multiples of 0.2 <br>
    "loguniform": tune.loguniform(1e-4, 1e-1),  # Uniform float in log space <br>
    "qloguniform": tune.qloguniform(1e-4, 1e-1, 5e-5),  # Round to multiples of 0.00005 <br>
    "randn": tune.randn(10, 2),  # Normal distribution with mean 10 and sd 2 <br>
    "qrandn": tune.qrandn(10, 2, 0.2),  # Round to multiples of 0.2 <br>
    "randint": tune.randint(-9, 15),  # Random integer between -9 and 15 <br>
    "qrandint": tune.qrandint(-21, 12, 3),  # Round to multiples of 3 (includes 12) <br>
    "lograndint": tune.lograndint(1, 10),  # Random integer in log space <br>
    "qlograndint": tune.qlograndint(1, 10, 2),  # Round to multiples of 2 <br>
    "choice": tune.choice(["a", "b", "c"]),  # Choose one of these options uniformly <br>
    "func": tune.sample_from( lambda spec: spec.config.uniform * 0.01 ),  # Depends on other value <br>
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)) # Return random integers from low (inclusive) to high (exclusive). <br>
    "grid": tune.grid_search([32, 64, 128]),  # Search over all these values <br>
} <br>

### Trainable

--> Trainable is an object that is passed to tune.
--> There are two options.
#### a) Function API:- 


```
from ray import tune
from ray.air import session

def trainable_func(config: dict):
    intermediate_score = 0
    for x in range(20):
        intermediate_score = objective(x, config["a"], config["b"])
        session.report({"score": intermediate_score})  # This sends the score to Tune.


tuner = tune.Tuner(trainable_func, param_space={"a": 2, "b": 4})
results = tuner.fit()  
```

#### b) Class API:- 

```
from ray import air, tune

class TrainableCls(tune.Trainable):
    def setup(self, config: dict):
        # config (dict): A dict of hyperparameters
        self.x = 0
        self.a = config['a']
        self.b = config['b']

    def step(self):  # This is called iteratively.
        score = objective(self.x, self.a, self.b)
        self.x += 1
        return {"score": score}

tuner = tune.Tuner(
    TrainableCls,
    run_config=air.RunConfig(
        # Train for 20 steps
        stop={"training_iteration": 20},
        checkpoint_config=air.CheckpointConfig(
            # We haven't implemented checkpointing yet. See below!
            checkpoint_at_end=False
        ),
    ),
    param_space={"a": 2, "b": 4},
)
results = tuner.fit()
```

### Search Algorithm

--> Search Algorithm suggests hyperparameter configurations. If nothing is specified random search is used by default.

--> Different Search Algorithms
1) BasicVariantGenerator
The BasicVariantGenerator is used per default if no search algorithm is passed to Tuner. It is mainly used for Random search and grid search.

```
from ray import tune

# This will automatically use the `BasicVariantGenerator`
tuner = tune.Tuner(
    lambda config: config["a"] + config["b"],
    tune_config=tune.TuneConfig(
        num_samples=4
    ),
    param_space={
        "a": tune.grid_search([1, 2]),
        "b": tune.randint(0, 3)
    },
)
tuner.fit()
```

In the example above, 8 trials will be generated: For each sample (4), each of the grid search variants for a will 
be sampled once. The b parameter will be sampled randomly.


2) AxSearch

Ax is a platform for understanding, managing, deploying, and automating adaptive experiments. Ax provides an easy to use 
interface with BoTorch, a flexible, modern library for Bayesian optimization in PyTorch.


```
from ray import tune
from ray.tune.search.ax import AxSearch

config = {
    "x1": tune.uniform(0.0, 1.0),
    "x2": tune.uniform(0.0, 1.0)
}

def easy_objective(config):
        result = train_model(config["x1"] , config["x2"])
        tune.report(score=result)

ax_search = AxSearch(metric="score", mode="max")
tuner = tune.Tuner(
    easy_objective,
    tune_config=tune.TuneConfig(
        search_alg=ax_search,
    ),
    param_space=config,
)
tuner.fit()
```


3) BayesOptSearch

BayesOpt is a constrained global optimization package utilizing Bayesian inference on gaussian processes, where the emphasis is on finding the maximum value of 
an unknown function in as few iterations as possible. 

```
# pip install bayesian-optimization
from ray.tune.search.bayesopt import BayesOptSearch
from ray import air

# Define the search space
search_space = {"a": tune.uniform(0, 1), "b": tune.uniform(0, 20)}

algo = BayesOptSearch(random_search_steps=4)

tuner = tune.Tuner(
    trainable_func,
    tune_config=tune.TuneConfig(
        metric="score",
        mode="min",
        search_alg=algo,
    ),
    run_config=air.RunConfig(stop={"training_iteration": 20}),
    param_space=search_space,
)
tuner.fit()
```

```
Repeater to average over multiple evaluations of the same hyperparameter configurations. 
This is useful in cases where the evaluated training procedure has high variance (i.e., in reinforcement learning).

from ray.tune.search import Repeater

search_alg = BayesOptSearch(...)
re_search_alg = Repeater(search_alg, repeat=10)

# Repeat 2 samples 10 times each.
tuner = tune.Tuner(
    trainable,
    tune_config=tune.TuneConfig(
        search_alg=re_search_alg,
        num_samples=20,
    ),
)
tuner.fit()

```

4) Optuna

```
from ray.tune.search.optuna import OptunaSearch

config = {
    "a": tune.uniform(6, 8)
    "b": tune.loguniform(1e-4, 1e-2)
}

optuna_search = OptunaSearch(
    metric="loss",
    mode="min")

tuner = tune.Tuner(
    trainable,
    tune_config=tune.TuneConfig(
        search_alg=optuna_search,
    ),
    param_space=config,
)
tuner.fit()
```

5) Scikit-Optimize

```
config = {
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100)
}

skopt_search = SkOptSearch(
    metric="mean_loss",
    mode="min")

tuner = tune.Tuner(
    trainable_function,
    tune_config=tune.TuneConfig(
        search_alg=skopt_search
    ),
    param_space=config
)
tuner.fit()
```

6) HyperOpt

```
from ray.tune.search.hyperopt import HyperOptSearch

# Define the search space
search_space = {"a": tune.uniform(0, 1), "b": tune.uniform(0, 20)}

algo = HyperOptSearch()

tuner = tune.Tuner(
    trainable_func,
    tune_config=tune.TuneConfig(
        metric="score",
        mode="min",
        search_alg=algo,
    ),
    param_space=search_space,
)
tuner.fit()
```

### Scheduler

--> schedulers can stop, pause, or tweak the hyperparameters of running trials, potentially making your hyperparameter tuning process much faster.

```
from ray.tune.schedulers import HyperBandScheduler

# Create HyperBand scheduler and minimize the score
hyperband = HyperBandScheduler(metric="score", mode="max")

config = {"a": tune.uniform(0, 1), "b": tune.uniform(0, 1)}

tuner = tune.Tuner(
    trainable_func,
    tune_config=tune.TuneConfig(
        num_samples=20,
        scheduler=hyperband,
    ),
    param_space=config,
)
tuner.fit()
```

--> Other example of schedulers are <br>
1) ASHAScheduler
2) HyperBandScheduler

### Trials
--> Tuner.fit execute and manage hyperparameters tuning and generate trials
Example_1:
space = {"a": tune.uniform(0, 1), "b": tune.uniform(0, 1)}
tuner = tune.Tuner(
    trainable_func, 
    param_space=space, 
    tune_config=tune.TuneConfig(num_samples=10)
)
tuner.fit()

1) trainable_func --> the function with config parameter
2) param_space --> the hyperparameter search space
3) TuneConfig: <br>
    3.a) <b>num_samples</b> - total number of trials to run. Tune automatically determines how many trials will run in parallel
    You can specify the cores. <br>
    3.b) <b>max_concurrent_trials</b> - It specifies the max number of trials to run concurrently
4) resources:
    it specifies resources ~ cpu/gpu to run the trials
  
```
trainable_with_resources = tune.with_resources(trainable_func,
    resources=lambda spec: {"gpu": 1} if spec.config.use_gpu else {"cpu": 4})
tuner = tune.Tuner(
    trainable_with_resources,
    tune_config=tune.TuneConfig(
                                num_samples=10,
                                max_concurrent_trials=10,)
)
results = tuner.fit()
```

### Analyses

--> Tuner.fit() returns an ResultGrid object which has methods you can use for analyzing your training.

```
tuner = tune.Tuner(
    trainable_func,
    tune_config=tune.TuneConfig(
        metric="score",
        mode="min",
        search_alg=BayesOptSearch(random_search_steps=4),
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 20},
    ),
    param_space=config,
)
results = tuner.fit()

best_result = results.get_best_result()  # Get best result object
best_config = best_result.config  # Get best trial's hyperparameters
best_logdir = best_result.log_dir  # Get best trial's logdir
best_checkpoint = best_result.checkpoint  # Get best trial's best checkpoint
best_metrics = best_result.metrics  # Get best trial's last results
best_result_df = best_result.metrics_dataframe  # Get best result as pandas dataframe
```

In [24]:
import traceback

from sklearn.model_selection import train_test_split
import pandas as pd

import shutil
import json
import pickle5 as pickle

import os
from ray.tune import TuneConfig

import numpy as np
from ray.air import session

from collections import OrderedDict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from functools import partial

from ray.air.config import RunConfig

from ray import tune
from ray.tune import Tuner, TuneConfig
from ray import air
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.bayesopt import BayesOptSearch

In [2]:
'''
Custom pytorch model
'''

class Net(nn.Module):
    
    def __init__(self, conf):
        super(Net, self).__init__()
        self.net_layer = nn.Sequential(OrderedDict([
            ('1_LINEAR_LAYER', nn.Linear(8, 4)),
            ('2_LINEAR_LAYER', nn.Linear(4, 1))
        ]))

    def forward(self, x):
        
        layer_1 = self.net_layer._modules['1_LINEAR_LAYER']
        layer_1_op = layer_1(x)
        
        layer_2 = self.net_layer._modules['2_LINEAR_LAYER']
        layer_2_op = layer_2(layer_1_op)
        
        op = torch.sigmoid(layer_2_op)
        
        return op

## Data load <a class="anchor" id="data_load"></a>
<b>Data:</b> <br>
https://www.kaggle.com/datasets/whenamancodes/predict-diabities <br>
-- selected three features ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'] <br>
-- target is 'Outcome' ~ Diabetes (1 - yes), (0 - no) <br>

In [3]:
def get_conf():
    try:
        conf = {
            "data_fl_path": "../DataSets/diabetes.csv",
            "data_ratio":{
                "train_ratio":0.8,
                "test_ratio":0.1,
                "valid_ratio":0.1 
            },
            "device": "cpu",
            "epochs": 10,
            "lr": 0.00001,
            "momentum": 0.9,
            "model_nm": "diabetes_model.pth",
            "model_path": "../Models/"
        }     
        return conf
    except Exception as e:
        raise e

In [4]:
class DiabetesDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values,dtype=torch.float32)
        self.y = torch.tensor(y.values,dtype=torch.float32)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
class DataEngine:
    def __init__(self, conf):
        self.df = pd.read_csv(conf['data_fl_path'])
        self.df = self.df.dropna()
        
    def split_data(self, conf):
        try:
            X = self.df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
            y = self.df[['Outcome']]

            X_train, X_rem, y_train, y_rem = train_test_split(X,y, stratify=y, train_size=conf['data_ratio']['train_ratio']) #0.8

            X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, stratify=y_rem, test_size=0.5) #0.1, 0.1

            print("X_train: " , X_train.shape, "y_train: " , y_train.shape)
            print("X_valid: " , X_valid.shape, "y_valid: " , y_valid.shape)
            print("X_test: " , X_test.shape, "y_test: " , y_test.shape)

            data = {
                "X_train": X_train,
                "y_train": y_train,
                "X_valid": X_valid,
                "y_valid": y_valid,
                "X_test": X_test,
                "y_test": y_test
            }

            return data

        except Exception as e:
            raise e

In [6]:
class ModelEngine:
    def __init__(self, conf):
        self.net = Net(conf)
        self.net.to(conf["device"])
        self.optimizer = optim.SGD(self.net.parameters(), lr=conf["lr"], momentum=conf["momentum"])
        
    def loss_fn(self, outputs, targets):
        loss = nn.BCELoss()(outputs, targets)
        return loss
    
    def train(self, db_dataloader, conf):     
        # move the model into train mode
        self.net.train()  
        train_loss = 0.0
        for X_sample, y_sample in db_dataloader:
            X_sample, y_sample = X_sample.to(conf["device"]), y_sample.to(conf["device"])
            
            # zero the parameter gradients
            self.optimizer.zero_grad()
            
            outputs = self.net(X_sample)
            
            loss = self.loss_fn(outputs, y_sample)
            loss.backward()
            self.optimizer.step()
            
            train_loss += loss.item()
            
        train_loss = round(train_loss/len(db_dataloader), 5)
        
        return train_loss
        
    def evaluate(self, db_dataloader, conf):
        # move the model into eval mode
        self.net.eval()
        test_loss = 0.0
        for X_sample, y_sample in db_dataloader:

            X_sample, y_sample = X_sample.to(conf["device"]), y_sample.to(conf["device"])

            outputs = self.net(X_sample)

            loss = self.loss_fn(outputs, y_sample)

            test_loss += loss.item()
            
        test_loss = round(test_loss/len(db_dataloader), 5)
        
        return test_loss
    
    def save_model(self, conf, ind):
        torch.save(self.net.state_dict(), ind+conf['model_nm'])

In [7]:
def train_func(config,
               db_train_dataloader=None, db_valid_dataloader=None, conf=None):
    try:
        conf["lr"] = config["lr"]
        conf["momentum"] = config["momentum"]
        modelEngg = ModelEngine(conf)
        total_loss = 0
        for epoch in range(conf["epochs"]+1):
            train_loss = modelEngg.train(db_train_dataloader, conf)
            valid_loss = modelEngg.evaluate(db_valid_dataloader, conf)
            
            print(f'Epoch:- {epoch}  TrainLoss:- {train_loss}  ValidLoss:- {valid_loss}')
            total_loss += train_loss
            
            if epoch % 5 == 0:
                # This saves the model to the trial directory
                ind = "epoch_"+str(epoch)+"_"
                modelEngg.save_model(conf, ind)
        
        # This saves the model to the trial directory
        ind = "final_"
        modelEngg.save_model(conf, ind) 
        # tune.report(loss=total_loss/conf["epochs"])
        session.report({"loss": total_loss/conf["epochs"]})
    except Exception as e:
        raise e

In [8]:
# class TrainableCls(tune.Trainable):
    
#     # setup function is invoked once training starts.
#     def setup(self, config: dict, db_train_dataloader=None, db_valid_dataloader=None, conf=None):
#         # config (dict): A dict of hyperparameters
#         self.db_train_dataloader = db_train_dataloader
#         self.db_valid_dataloader = db_valid_dataloader
#         self.conf = conf
#         self.lr = config["lr"]

#     # step function is called iteratively. Each time, the Trainable object executes one logical iteration of training in the tuning process
#     def step(self):
#         self.conf["lr"] = self.lr
#         loss = train_func(self.db_train_dataloader, self.db_valid_dataloader, self.conf) #objective function
#         return {"loss": loss}

In [9]:
def hyperparam_tune_random(db_train_dataloader, db_valid_dataloader, conf):
    try:
        # defining search space
        # config = {
        #     "lr": tune.sample_from(lambda spec: 10 ** (-10 * np.random.rand()))
        # }
        
        config = {
            "lr": tune.loguniform(1e-4, 1e-2),
            "momentum": tune.choice([0.7, 0.8, 0.9])
        }
        
        # defining search algorithm and metric
        # bayesopt = BayesOptSearch(random_search_steps=4)
        
        scheduler = ASHAScheduler(
                        metric="loss",
                        mode="min",
                        max_t=10,
                        grace_period=1,
                        reduction_factor=2)
        
        reporter = CLIReporter(
                # parameter_columns=["l1", "l2", "lr", "batch_size"],
                metric_columns=["loss", "training_iteration"])
        
        result = tune.run(
            partial(train_func, db_train_dataloader=db_train_dataloader, db_valid_dataloader=db_valid_dataloader, conf=conf),
            config=config,
            num_samples=10,
            scheduler=scheduler,
            progress_reporter=reporter
        )

        # results = tuner.fit()
        
        # best_trial = results.get_best_result() 
        best_trial = result.get_best_trial("loss", "min", "last")
        print("Best trial config: {}".format(best_trial.config))
        
        return best_trial
        
    except Exception as e:
        raise e

In [10]:
def hyperparam_tune_bayes_opt(db_train_dataloader, db_valid_dataloader, conf):
    try:        
        param_space = {
            "lr": tune.loguniform(1e-4, 1e-2),
            "momentum": tune.uniform(0.6, 0.9),
        }
        bayesopt = BayesOptSearch(metric="loss", mode="min")
        tune_config = TuneConfig(
                            max_concurrent_trials=10,
                            num_samples=10,
                            search_alg=bayesopt,
                        )
        
        tuner = Tuner(
                trainable=tune.with_parameters(train_func, 
                                               db_train_dataloader=db_train_dataloader, 
                                               db_valid_dataloader=db_valid_dataloader, 
                                               conf=conf),
                run_config=RunConfig(name="bayesopt_tuner", local_dir="~/ray_results"),
                param_space=param_space,
                tune_config=tune_config
            )

        results = tuner.fit()
        
        best_trial = results.get_best_result(metric="loss", mode="min") 
        # best_trial = result.get_best_trial("loss", "min", "last")
        print("Best trial config: {}".format(best_trial.config))
        
        return best_trial
        
    except Exception as e:
        raise e

In [11]:
def save_best_model(best_trial, conf):
    try:
        logdir = best_trial.log_dir
        
        ind = "final_"
        state_dict = torch.load(os.path.join(logdir, ind+conf['model_nm']))
        
        conf["lr"] = best_trial.config["lr"]
        conf["momentum"] = best_trial.config["momentum"]
        
        net = Net(conf)
        net.load_state_dict(state_dict)
        
        torch.save(net.state_dict(), conf['model_path']+conf['model_nm'])
        
        origin = os.path.join(logdir, "params.json")
        target = conf['model_path']+"params.json"
        # copying best parameters
        shutil.copy(origin, target)
        
        origin = os.path.join(logdir, "params.pkl")
        target = conf['model_path']+"params.pkl"
        # copying best parameters
        shutil.copy(origin, target)
        
    except Exception as e:
        raise e

In [22]:
def test_func(db_test_dataloader, conf):
    try:
        conf["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load(conf['model_path']+conf['model_nm'])
        
        with open(conf['model_path']+'params.json', 'r') as f:
            best_result = json.load(f)
        
        conf["lr"] = best_result["lr"]
        conf["momentum"] = best_result["momentum"]
        
        print(conf)
        
        modelEngg = ModelEngine(conf)
        
        test_loss = modelEngg.evaluate(db_test_dataloader, conf)
        
        print(f'test_loss:- {test_loss}')
        
    except Exception as e:
        raise e

In [20]:
def main():
    try:
        # get conf
        conf = get_conf()
        conf["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # loading the data
        dataEng = DataEngine(conf)
        
        # splitting the data
        db_df_split = dataEng.split_data(conf)
        
        # converting data to dataloder
        db_train_data = DiabetesDS(db_df_split["X_train"], db_df_split["y_train"])
        db_train_dataloader = DataLoader(db_train_data, batch_size=5, shuffle=False, num_workers=1)
        
        db_valid_data = DiabetesDS(db_df_split["X_valid"], db_df_split["y_valid"])
        db_valid_dataloader = DataLoader(db_valid_data, batch_size=5, shuffle=False, num_workers=1)
        
        db_test_data = DiabetesDS(db_df_split["X_test"], db_df_split["y_test"])
        db_test_dataloader = DataLoader(db_test_data, batch_size=5, shuffle=False, num_workers=1)
        
        
        # training the model
        # train_func(db_train_dataloader, db_valid_dataloader, conf) 
        
        # hyper parameter tuning
        # best_trial = hyperparam_tune_random(db_train_dataloader, db_valid_dataloader, conf)
        best_trial = hyperparam_tune_bayes_opt(db_train_dataloader, db_valid_dataloader, conf)
        
        # saving the best model for further use
        save_best_model(best_trial, conf)
        
        # test with best model
        test_func(db_test_dataloader, conf)
        
    except Exception as e:
        traceback.print_exc()
    finally:
        print("DONE")

In [25]:
if __name__=="__main__":
    main()

X_train:  (614, 8) y_train:  (614, 1)
X_valid:  (77, 8) y_valid:  (77, 1)
X_test:  (77, 8) y_test:  (77, 1)


2023-01-21 14:54:24,061	WARNING bayesopt_search.py:425 -- BayesOpt does not support specific sampling methods. The LogUniform sampler will be dropped.
2023-01-21 14:54:24,069	WARNING bayesopt_search.py:425 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.


(train_func pid=31339) Epoch:- 0  TrainLoss:- 37.12057  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 1  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 2  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 3  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 4  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 5  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 6  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 7  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 8  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 9  TrainLoss:- 34.8374  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 10  TrainLoss:- 34.8374  ValidLoss:- 37.5


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_func_c92dec0c,2023-01-21_14-54-43,True,,83178d67ea1f40a8aef669c1784e2718,"1_lr=0.0038,momentum=0.8852",JAYDEEPs-MacBook-Pro.local,1,38.5495,127.0.0.1,31339,6.51219,6.51219,6.51219,1674334483,0,,1,c92dec0c,0.00585079
train_func_d0c767ea,2023-01-21_14-54-54,True,,cf2fd35482ac43d59248027f8e81275f,"2_lr=0.0073,momentum=0.7796",JAYDEEPs-MacBook-Pro.local,1,37.9517,127.0.0.1,31347,9.88843,9.88843,9.88843,1674334494,0,,1,d0c767ea,0.00598717
train_func_d55edfae,2023-01-21_14-55-09,True,,3ba37c720b3e439aa55810e20a4d3163,"3_lr=0.0016,momentum=0.6468",JAYDEEPs-MacBook-Pro.local,1,0.888123,127.0.0.1,31375,13.1439,13.1439,13.1439,1674334509,0,,1,d55edfae,0.00909495
train_func_d5698aa8,2023-01-21_14-54-52,True,,83178d67ea1f40a8aef669c1784e2718,"4_lr=0.0007,momentum=0.8599",JAYDEEPs-MacBook-Pro.local,1,0.892541,127.0.0.1,31339,8.2805,8.2805,8.2805,1674334492,0,,1,d5698aa8,0.00585079
train_func_d57396ce,2023-01-21_14-55-09,True,,7780d34da54341cab9176f20dd8f921a,"5_lr=0.0061,momentum=0.8124",JAYDEEPs-MacBook-Pro.local,1,70.6311,127.0.0.1,31377,13.5841,13.5841,13.5841,1674334509,0,,1,d57396ce,0.00447702
train_func_d57902bc,2023-01-21_14-55-07,True,,cf2fd35482ac43d59248027f8e81275f,"6_lr=0.0003,momentum=0.8910",JAYDEEPs-MacBook-Pro.local,1,1.00031,127.0.0.1,31347,11.0359,11.0359,11.0359,1674334507,0,,1,d57902bc,0.00598717
train_func_dc52e7ec,2023-01-21_14-55-07,True,,83178d67ea1f40a8aef669c1784e2718,"7_lr=0.0083,momentum=0.6637",JAYDEEPs-MacBook-Pro.local,1,38.2234,127.0.0.1,31339,11.1355,11.1355,11.1355,1674334507,0,,1,dc52e7ec,0.00585079
train_func_dc5f58e2,2023-01-21_14-55-16,True,,cf2fd35482ac43d59248027f8e81275f,"8_lr=0.0019,momentum=0.6550",JAYDEEPs-MacBook-Pro.local,1,0.866063,127.0.0.1,31347,8.74702,8.74702,8.74702,1674334516,0,,1,dc5f58e2,0.00598717
train_func_e2f70dee,2023-01-21_14-55-16,True,,83178d67ea1f40a8aef669c1784e2718,"9_lr=0.0031,momentum=0.7574",JAYDEEPs-MacBook-Pro.local,1,0.869702,127.0.0.1,31339,8.76063,8.76063,8.76063,1674334516,0,,1,e2f70dee,0.00585079
train_func_e3137c22,2023-01-21_14-55-16,True,,3ba37c720b3e439aa55810e20a4d3163,"10_lr=0.0044,momentum=0.6874",JAYDEEPs-MacBook-Pro.local,1,0.987656,127.0.0.1,31375,7.48505,7.48505,7.48505,1674334516,0,,1,e3137c22,0.00909495


(train_func pid=31339) Epoch:- 0  TrainLoss:- 2.68961  ValidLoss:- 0.6147
(train_func pid=31339) Epoch:- 1  TrainLoss:- 0.63117  ValidLoss:- 0.61167
(train_func pid=31339) Epoch:- 2  TrainLoss:- 0.62833  ValidLoss:- 0.61231
(train_func pid=31347) Epoch:- 0  TrainLoss:- 32.17545  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 3  TrainLoss:- 0.62682  ValidLoss:- 0.61203
(train_func pid=31347) Epoch:- 1  TrainLoss:- 34.73417  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 4  TrainLoss:- 0.62545  ValidLoss:- 0.61153
(train_func pid=31347) Epoch:- 2  TrainLoss:- 34.73417  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 5  TrainLoss:- 0.62406  ValidLoss:- 0.61104
(train_func pid=31347) Epoch:- 3  TrainLoss:- 34.73417  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 6  TrainLoss:- 0.62266  ValidLoss:- 0.61058
(train_func pid=31347) Epoch:- 4  TrainLoss:- 34.73417  ValidLoss:- 37.5
(train_func pid=31339) Epoch:- 7  TrainLoss:- 0.62128  ValidLoss:- 0.61015
(train_func pid=31347) Epoch:- 5  Tr

2023-01-21 14:55:17,170	INFO tune.py:778 -- Total run time: 53.17 seconds (52.79 seconds for the tuning loop).


Best trial config: {'lr': 0.0019000671753502962, 'momentum': 0.6550213529560301}
{'data_fl_path': '../DataSets/diabetes.csv', 'data_ratio': {'train_ratio': 0.8, 'test_ratio': 0.1, 'valid_ratio': 0.1}, 'device': device(type='cpu'), 'epochs': 10, 'lr': 0.0019000671753502962, 'momentum': 0.6550213529560301, 'model_nm': 'diabetes_model.pth', 'model_path': '../Models/'}
test_loss:- 50.16075
DONE


## Resources
1) https://docs.ray.io/en/latest/tune/index.html